In [1]:
# importar paquetes a ocupar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet
import os

In [2]:
# importar archivos csv# Abrir el archivo comprimido
crossBattlelog = pd.read_parquet('datasets/teams/battlelog_teams.parquet')
crossBattlelog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 757358 entries, 0 to 757357
Data columns (total 44 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   battleTime                             757358 non-null  datetime64[ns]
 1   playertag                              757358 non-null  object        
 2   event_mode                             757358 non-null  category      
 3   event_map                              757358 non-null  category      
 4   battle_type                            757358 non-null  category      
 5   battle_result                          757358 non-null  category      
 6   battle_duration                        757358 non-null  Int16         
 7   battle_trophyChange                    573018 non-null  Int8          
 8   battle_team1_player1_tag               757358 non-null  object        
 9   battle_team1_player1_name              757356 no

In [3]:
# función para identificar a que team pertenece el player
def team_assignment(row):
    if any(row['playertag'] == tag for tag in [row.battle_team1_player1_tag, row.battle_team1_player2_tag, row.battle_team1_player3_tag]):
        return 'team1'
    elif any(row['playertag'] == tag for tag in [row.battle_team2_player1_tag, row.battle_team2_player2_tag, row.battle_team2_player3_tag]):
        return 'team2'
    else:
        return "error in team assignation for id " + str(row.name)

crossBattlelog['player_team'] = crossBattlelog.apply(team_assignment, axis=1)

crossBattlelog['player_team'].value_counts()

team2    383292
team1    374066
Name: player_team, dtype: int64

In [4]:
# crear columna winner_team
def winner_assignment(row):
	if row.battle_result == 'draw':
		return 'draw'
	elif (row.player_team == 'team1' and row.battle_result == 'victory') or (row.player_team == 'team2' and row.battle_result == 'defeat'):
		return 'team1'
	elif (row.player_team == 'team2' and row.battle_result == 'victory') or (row.player_team == 'team1' and row.battle_result == 'defeat'):
		return 'team2'
	else:
		return "error in winner assignation for id " + str(row.name)

crossBattlelog['winner_team'] = crossBattlelog.apply(winner_assignment, axis=1)

crossBattlelog['winner_team'].value_counts()

team1    376757
team2    375558
draw       5043
Name: winner_team, dtype: int64

In [5]:
# agregar columna identificando batallas low_trophy

# def check_low_trophy(val, threshold=250):
# 	if val < threshold:
# 		return True
# 	else:
# 		return False

# crossBattlelog['low_trophy'] = False

cols = ['battle_team1_player1_brawler_trophies', 'battle_team1_player2_brawler_trophies', 'battle_team1_player3_brawler_trophies', 'battle_team2_player1_brawler_trophies', 'battle_team2_player2_brawler_trophies', 'battle_team2_player3_brawler_trophies']

# for item in(cols):
# 	for i, trophies in enumerate(crossBattlelog[item]):
# 		if crossBattlelog.loc[i, 'low_trophy'] == False:
# 			crossBattlelog.loc[i, 'low_trophy'] = check_low_trophy(trophies)

crossBattlelog['low_trophy'] = crossBattlelog[cols].apply(lambda x: any(x < 250), axis=1)

crossBattlelog['low_trophy'].value_counts()

False    510962
True     246396
Name: low_trophy, dtype: int64

In [6]:
#subset low_trophy = False
crossBattlelog = crossBattlelog[crossBattlelog['low_trophy'] == False].reset_index(drop=True)

In [7]:
# crear columna winner
def subset_winner(df, team):

	df['winner'] = 'no'

	for i, winner_team in enumerate(df['winner_team']):
		if winner_team == team:
			df.loc[i, 'winner'] = 'yes'

	df = df.rename(columns={
		'battle_' + team + '_player1_brawler_name': 'battle_player1_brawler_name'
		,'battle_' + team + '_player1_brawler_power': 'battle_player1_brawler_power'
		,'battle_' + team + '_player1_brawler_trophies': 'battle_player1_brawler_trophies'
		,'battle_' + team + '_player2_brawler_name': 'battle_player2_brawler_name'
		,'battle_' + team + '_player2_brawler_power': 'battle_player2_brawler_power'
		,'battle_' + team + '_player2_brawler_trophies': 'battle_player2_brawler_trophies'
		,'battle_' + team + '_player3_brawler_name': 'battle_player3_brawler_name'
		,'battle_' + team + '_player3_brawler_power': 'battle_player3_brawler_power'
		,'battle_' + team + '_player3_brawler_trophies': 'battle_player3_brawler_trophies'
	})

	df = df[[
		'battleTime'
		,'event_mode'
		,'event_map'
		,'battle_type'
		,'battle_duration'
		,'battle_player1_brawler_name'
		,'battle_player1_brawler_power'
		,'battle_player1_brawler_trophies'
		,'battle_player2_brawler_name'
		,'battle_player2_brawler_power'
		,'battle_player2_brawler_trophies'
		,'battle_player3_brawler_name'
		,'battle_player3_brawler_power'
		,'battle_player3_brawler_trophies'
		,'winner'
	]]

	return df

def clean_winner(team):

	if team == 'team1':
		oteam = 'team2'
	else:
		oteam = 'team1'
	
	subset = crossBattlelog[crossBattlelog['winner_team'] == team].reset_index(drop=True)

	dfwinners = subset_winner(subset, team)

	dfloosers = subset_winner(subset, oteam)

	return pd.concat([dfwinners, dfloosers], ignore_index=True)

team1 = clean_winner('team1')
team2 = clean_winner('team2')

battlelogProcess = pd.concat([team1, team2], ignore_index=True)
battlelogProcess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014226 entries, 0 to 1014225
Data columns (total 15 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   battleTime                       1014226 non-null  datetime64[ns]
 1   event_mode                       1014226 non-null  category      
 2   event_map                        1014226 non-null  category      
 3   battle_type                      1014226 non-null  category      
 4   battle_duration                  1014226 non-null  Int16         
 5   battle_player1_brawler_name      1014226 non-null  category      
 6   battle_player1_brawler_power     1014226 non-null  Int8          
 7   battle_player1_brawler_trophies  1014226 non-null  Int16         
 8   battle_player2_brawler_name      1014226 non-null  category      
 9   battle_player2_brawler_power     1014226 non-null  Int8          
 10  battle_player2_brawler_trophie

In [8]:
# unificar bases de brawlers
bk1 = battlelogProcess.rename(columns={'battle_player1_brawler_name': 'brawlername'})
bk2 = battlelogProcess.rename(columns={'battle_player2_brawler_name': 'brawlername'})
bk3 = battlelogProcess.rename(columns={'battle_player3_brawler_name': 'brawlername'})

brawler = pd.concat([
bk1[['battleTime','event_mode','event_map','brawlername','winner']], 
bk2[['battleTime','event_mode','event_map','brawlername','winner']], 
bk3[['battleTime','event_mode','event_map','brawlername','winner']]
], ignore_index=True)

brawler

,battleTime,event_mode,event_map,brawlername,winner
0,2023-02-11,gemGrab,Deep Diner,MAX,yes
1,2023-02-11,gemGrab,Deep Diner,MAX,yes
2,2023-02-11,gemGrab,Deep Diner,MAX,yes
3,2023-02-11,gemGrab,Deep Diner,GRIFF,yes
4,2023-02-11,gemGrab,Deep Diner,MAX,yes
...,...,...,...,...,...
3042673,2023-01-02,hotZone,Quarter Pounder,BULL,no
3042674,2023-01-01,hotZone,Quarter Pounder,BEA,no
3042675,2023-01-02,gemGrab,Double Swoosh,SPIKE,no
3042676,2023-01-02,gemGrab,Double Swoosh,BROCK,no


In [9]:
brawler['winner'].value_counts()

yes    1521339
no     1521339
Name: winner, dtype: int64

In [10]:
brawler.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3042678 entries, 0 to 3042677
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   battleTime   datetime64[ns]
 1   event_mode   category      
 2   event_map    category      
 3   brawlername  category      
 4   winner       object        
dtypes: category(3), datetime64[ns](1), object(1)
memory usage: 55.1+ MB


In [11]:
# crear pivot tables
pivot_brawler = brawler.pivot_table(index=['battleTime','brawlername','event_mode','event_map'], columns='winner', aggfunc='size').reset_index()
pivot_brawler

winner,battleTime,brawlername,event_mode,event_map,no,yes
0,2022-09-06,8-BIT,bounty,Backyard Bowl,0,0
1,2022-09-06,8-BIT,bounty,Beach Ball,0,0
2,2022-09-06,8-BIT,bounty,Belle's Rock,0,0
3,2022-09-06,8-BIT,bounty,Bridge Too Far,0,0
4,2022-09-06,8-BIT,bounty,Canal Grande,0,0
...,...,...,...,...,...,...
1225723,2023-02-11,TICK,knockout,Super Beach,0,0
1225724,2023-02-11,TICK,knockout,Temple Ruins,0,0
1225725,2023-02-11,TICK,knockout,Tornado Ring,0,0
1225726,2023-02-11,TICK,knockout,Undermine,0,0


In [12]:
# agregar columna totales

pivot_brawler['total_games'] = pivot_brawler['no'] + pivot_brawler['yes']

pivot_brawler = pivot_brawler.sort_values(by=['brawlername','battleTime','event_mode','event_map'], ascending=False).reset_index(drop=True)

pivot_brawler = pivot_brawler[pivot_brawler['total_games'] > 0].reset_index(drop=True)

pivot_brawler

winner,battleTime,brawlername,event_mode,event_map,no,yes,total_games
0,2023-02-11,TICK,knockout,Flaring Phoenix,281,214,495
1,2023-02-11,TICK,hotZone,Controller Chaos,236,51,287
2,2023-02-11,TICK,gemGrab,Deep Diner,154,74,228
3,2023-02-11,TICK,brawlBall,Super Beach,115,84,199
4,2023-02-11,TICK,brawlBall,Sneaky Fields,40,27,67
...,...,...,...,...,...,...,...
22522,2022-12-27,8-BIT,gemGrab,Minecart Madness,2,0,2
22523,2022-12-26,8-BIT,gemGrab,Minecart Madness,2,0,2
22524,2022-10-30,8-BIT,heist,Safe Zone,1,0,1
22525,2022-10-15,8-BIT,gemGrab,Hard Rock Mine,0,2,2


In [13]:
# export brawler_by_event_map
pivot_brawler.to_csv('datasets/teams/brawler_by_event_map.csv')

In [14]:
# definición de funciones
# función crea cuadrantes entre dos columnas

def quadrants(col_a, col_b):
	
	a = (col_a / col_a.quantile(0.5)) >= 1
	b = (col_b / col_b.quantile(0.5)) >= 1

	return a, b

# función que crea distribución overall
def dist_complete():
	df_gm = pivot_brawler.groupby(['event_mode','event_map']).sum(numeric_only=True).reset_index()
	df = pivot_brawler.groupby(['brawlername','event_mode','event_map']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['yes'] / df['total_games']
	df = pd.merge(df, df_gm[['event_mode','event_map','total_games']], on=['event_mode','event_map'], how='left', suffixes=('', '_map'))
	df['userate'] = df['total_games'] / df['total_games_map']
	df['wr_top'], df['ur_top'] = quadrants(df['winrate'], df['userate'])
	df['percentile_quadrant'] = df.apply(lambda x: 'Q1' if x['wr_top'] and x['ur_top'] else ('Q2' if x['wr_top'] else ('Q3' if x['ur_top'] else 'Q4')), axis=1)
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

# función que crea distribución overall
def dist_overall():
	df = pivot_brawler.groupby(['brawlername']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['yes'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['wr_top'], df['ur_top'] = quadrants(df['winrate'], df['userate'])
	df['percentile_quadrant'] = df.apply(lambda x: 'Q1' if x['wr_top'] and x['ur_top'] else ('Q2' if x['wr_top'] else ('Q3' if x['ur_top'] else 'Q4')), axis=1)
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

# función que crea distribución por brawler general
def dist_gamemode(gamemode):
	df = pivot_brawler[(pivot_brawler['event_mode']==gamemode)].groupby(['brawlername','event_mode']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['yes'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['wr_top'], df['ur_top'] = quadrants(df['winrate'], df['userate'])
	df['percentile_quadrant'] = df.apply(lambda x: 'Q1' if x['wr_top'] and x['ur_top'] else ('Q2' if x['wr_top'] else ('Q3' if x['ur_top'] else 'Q4')), axis=1)
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

# función que crea distribución por brawler
def dist_brawler(brawler):
	df = pivot_brawler[(pivot_brawler['brawlername']==brawler)].groupby(['event_mode']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['yes'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

# función que crea distribución por gamemode y map
def dist_gamemode_map(gamemode, map):
	df = pivot_brawler[(pivot_brawler['event_mode']==gamemode) & (pivot_brawler['event_map']==map)].groupby(['brawlername']).sum(numeric_only=True).reset_index()
	df['winrate'] = df['yes'] / df['total_games']
	df['userate'] = df['total_games'] / sum(df['total_games'])
	df['wr_top'], df['ur_top'] = quadrants(df['winrate'], df['userate'])
	df['percentile_quadrant'] = df.apply(lambda x: 'Q1' if x['wr_top'] and x['ur_top'] else ('Q2' if x['wr_top'] else ('Q3' if x['ur_top'] else 'Q4')), axis=1)
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

# función que crea distribución por fecha
def dist_brawler_date(brawler, mindate):

	df_gm = pivot_brawler.groupby(['battleTime']).sum(numeric_only=True).reset_index()
	df = pivot_brawler[pivot_brawler['brawlername'] == brawler].groupby(['battleTime','brawlername']).sum(numeric_only=True).reset_index()
	df = df[df['battleTime'] >= pd.to_datetime(mindate)]
	df['winrate'] = df['yes'] / df['total_games']

	df = pd.merge(df, df_gm[['battleTime','total_games']], on='battleTime', how='left', suffixes=('', '_day'))

	df['userate'] = df['total_games'] / df['total_games_day']
	df = df[df['total_games'] > 0].reset_index(drop=True)
	return df

In [15]:
# export distribuition complete
dist_complete().to_parquet('datasets/teams/brawler_winrate_userate.parquet')
dist_overall().to_parquet('datasets/teams/brawler_winrate_userate_overall.parquet')

In [16]:
# definir estilo de plot
print(plt.style.available)
plt.style.use('seaborn-v0_8-dark-palette')

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [17]:
#estilo sns
print(sns.axes_style())
sns.set_style('darkgrid')
sns.set_palette('Accent')

{'axes.facecolor': 'white', 'axes.edgecolor': 'black', 'axes.grid': False, 'axes.axisbelow': 'line', 'axes.labelcolor': 'black', 'figure.facecolor': 'white', 'grid.color': '#b0b0b0', 'grid.linestyle': '-', 'text.color': 'black', 'xtick.color': 'black', 'ytick.color': 'black', 'xtick.direction': 'out', 'ytick.direction': 'out', 'lines.solid_capstyle': <CapStyle.projecting: 'projecting'>, 'patch.edgecolor': 'black', 'patch.force_edgecolor': False, 'image.cmap': 'viridis', 'font.family': ['sans-serif'], 'font.sans-serif': ['DejaVu Sans', 'Bitstream Vera Sans', 'Computer Modern Sans Serif', 'Lucida Grande', 'Verdana', 'Geneva', 'Lucid', 'Arial', 'Helvetica', 'Avant Garde', 'sans-serif'], 'xtick.bottom': True, 'xtick.top': False, 'ytick.left': True, 'ytick.right': False, 'axes.spines.left': True, 'axes.spines.bottom': True, 'axes.spines.right': True, 'axes.spines.top': True}


In [18]:
# winrate / userate overall
data = dist_overall()

fig, ax = plt.subplots(figsize=(8,8))

sns.scatterplot(
	data=data
	,x="winrate"
	,y="userate"
	,hue='percentile_quadrant'
	,ax=ax
)

for i, point in data.iterrows():
		ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername, fontsize=6.5)

yticks = ax.get_yticks()
xticks = ax.get_xticks()

ax.set_yticks(yticks)
ax.set_xticks(xticks)

ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

plt.title('Overall Winrate / Userate' + '\n' + 'Total Games: ' + str(sum(data['total_games'])))

plt.savefig('plots/overall/overall.png', dpi=300, bbox_inches='tight')

plt.close()

# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/overall/overall_q1.csv', index=False)

In [19]:
dist_gamemode('brawlBall').to_clipboard()

In [20]:
# winrate / userate
gamemodes = pivot_brawler['event_mode'].drop_duplicates().tolist()

for i, gm in enumerate(gamemodes):
    data = dist_gamemode(gm)

    fig, ax = plt.subplots(figsize=(8,7))

    sns.scatterplot(
        data=data
        ,x="winrate"
        ,y="userate"
        ,hue='percentile_quadrant'
        ,ax=ax
    )

    for i, point in data.iterrows():
        ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername, fontsize=6.5)

    yticks = ax.get_yticks()
    xticks = ax.get_xticks()

    ax.set_yticks(yticks)
    ax.set_xticks(xticks)

    ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
    ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])
    
    plt.title(str(data['event_mode'][0]) + '\n' + 'total games: ' + str(sum(data['total_games'])))

    plt.savefig('plots/gamemode/' + gm + '.png', dpi=300, bbox_inches='tight')
    
    plt.close()

    # data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/gamemode/' + gm + '.csv', index=False)

In [21]:
# winrate / userate
gamemodes = pivot_brawler['event_mode'].drop_duplicates().tolist()

for j, gm in enumerate(gamemodes):

	maps = pivot_brawler['event_map'][pivot_brawler['event_mode'] == gm].drop_duplicates().tolist()

	for i, map in enumerate(maps):
		data = dist_gamemode_map(gm, map)

		fig, ax = plt.subplots(figsize=(8,7))

		sns.scatterplot(
			data=data
			,x="winrate"
			,y="userate"
        	,hue='percentile_quadrant'
			,ax=ax
		)

		for i, point in data.iterrows():
			ax.text(point.winrate + 0.001, point.userate + 0.001, point.brawlername + '\n g:' + str(point.total_games) + ' w:' + str(point.yes), fontsize=6.5)

		yticks = ax.get_yticks()
		xticks = ax.get_xticks()

		ax.set_yticks(yticks)
		ax.set_xticks(xticks)

		ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
		ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

		plt.title(gm + '\n' + map + '\n' + 'total games: ' + str(sum(data['total_games'])))

		plt.savefig('plots/gamemode_map/' + gm + '_' + map + '.png', dpi=300, bbox_inches='tight')

		plt.close()

		# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/gamemode_map/' + gm + '_' + map + '.csv', index=False)

In [22]:
# winrate / userate by brawler
brawlers = pivot_brawler['brawlername'].drop_duplicates().tolist()

for j, br in enumerate(brawlers):

	data = dist_brawler(br)

	fig, ax = plt.subplots(figsize=(5,5))

	sns.scatterplot(
		data=data
		,x="winrate"
		,y="userate"
		,ax=ax
	)

	for i, point in data.iterrows():
		ax.text(point.winrate + 0.0002, point.userate + 0.0002, point[0] + '\n g:' + str(point.total_games) + ' w:' + str(point.yes), fontsize=6.5)

	yticks = ax.get_yticks()
	xticks = ax.get_xticks()

	ax.set_yticks(yticks)
	ax.set_xticks(xticks)

	ax.set_yticklabels(["{:,.00%}".format(y) for y in yticks])
	ax.set_xticklabels(["{:,.00%}".format(x) for x in xticks])

	plt.title(br + '\n' + 'total games: ' + str(sum(data['total_games'])))

	plt.savefig('plots/brawler/' + br + '.png', dpi=300, bbox_inches='tight')

	plt.close()

	# data.sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/brawlers/' + br + '.csv', index=False)

In [ ]:
# winrate / userate by date
brawlers = pivot_brawler['brawlername'].drop_duplicates().tolist()

for brawler in brawlers:

	data = dist_brawler_date(brawler, '2023-01-19')

	fig, ax = plt.subplots(2, 1, figsize=(7,5), sharex=False, sharey=False)

	ax[0].set_title('Overall Winrate / Userate' + '\n' + brawler)

	sns.lineplot(data=data, x='battleTime', y='winrate', ax=ax[0], color='g', label='winrate', marker='o')

	sns.barplot(data=data, x='battleTime', y='userate', ax=ax[1], color='b', label='userate')

	ax[0].set_xticklabels(labels='')
	ax[1].set_xticklabels(labels=data['battleTime'].dt.date, rotation=90)

	fig.legend()

	fig.savefig('plots/date/' + brawler + '.png', dpi=300, bbox_inches='tight')
	
	plt.close()

	# data[data['percentile_quadrant']=='Q1'].sort_values(by='winrate', ascending=False).to_csv('datasets/best_brawlers/overall/overall_q1.csv', index=False)